# *Model Training* dan Analisa Model

## Pengunduhan Data

Data yang akan kita gunakan berasal dari Kaggle.com dan akan kita unduh langsung ke dalam *notebook*. Pertama-tama kita akan siapkan Kaggle API untuk pengunduhan data.

In [ ]:
! pip install -q kaggle

from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"faisalkengo","key":"4de681ba9dea0e559d3e177f48270260"}'}

Kemudian kita akan buat direktori baru untuk menyimpan *file* Kaggle API yang sudah diunduh.

In [ ]:
 #Make directory named kaggle and copy kaggle.json file there.

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

Lalu kita akan ubah izin dari direktori yang dibuat.

In [ ]:
# Change the permissions of the file.

! chmod 600 ~/.kaggle/kaggle.json 

Setelah itu kita akan mengunduh *dataset* yang akan kita gunakan.

In [ ]:
! kaggle datasets download -d humanbojack/yolo-brand-object-detection  --unzip

 99% 1.76G/1.78G [01:05<00:00, 43.7MB/s]
100% 1.78G/1.78G [01:05<00:00, 29.2MB/s]


## Memuat *Library*

Sebelum kita memuat *library* yang akan digunakan, kita akan unduh model YOLOv5 yang akan kita gunakan untuk proses identifikasi gambar dan video. Selain itu kita juga akan memasang segala *library* yang digunakan di dalam model YOLOv5.

In [ ]:
! git clone https://github.com/ultralytics/yolov5
! pip install -qr /content/yolov5/requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 12751, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 12751 (delta 62), reused 74 (delta 39), pack-reused 12633
Receiving objects: 100% (12751/12751), 12.94 MiB | 5.20 MiB/s, done.
Resolving deltas: 100% (8771/8771), done.
     |████████████████████████████████| 1.6 MB 42.5 MB/s 


Kemudian kita akan muat *library* yang akan kita gunakan.

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

Sebelum menjalankan kode di bawah, kita akan ubah nama folder 'test' menjadi 'val' di images dan labels secara manual.

In [ ]:
img_data_path = '/content/640_data_augment_yolo_light/images'

path_labels_train = '/content/640_data_augment_yolo_light/labels/train'
path_labels_val = '/content/640_data_augment_yolo_light/labels/train'
path_images_train = '/content/640_data_augment_yolo_light/images/train'
path_images_val = '/content/640_data_augment_yolo_light/images/val'

os.mkdir('./yolov5/data/train')
os.mkdir('./yolov5/data/val')
os.mkdir('./yolov5/data/test')
os.mkdir('./yolov5/data/train/images')
os.mkdir('./yolov5/data/train/labels')
os.mkdir('./yolov5/data/test/images')
os.mkdir('./yolov5/data/test/labels')
os.mkdir('./yolov5/data/val/images')
os.mkdir('./yolov5/data/val/labels')

train_img = [*os.listdir(path_images_train)]
val_img = [*os.listdir(path_images_val)]
train_label = [*os.listdir(path_labels_train)]
val_label = [*os.listdir(path_labels_val)]

Lalu kita akan buat fungsi untuk membuka gambar yang akan diterapkan ke dalam model di dalam folder *train* dan *val*

In [ ]:
def copyImages(imageList, imagepath,folder_Name):
    for image in imageList:
        img = Image.open(imagepath+'/'+image)
        img1 = img.resize((640, 480))
        _ = img1.save("./yolov5/data/"+folder_Name+"/images/"+image)

copyImages(train_img, path_images_train,"train")
copyImages(val_img, path_images_val,"val")

Lalu kita akan buat fungsi untuk membuka label dari gambar yang akan diterapkan ke dalam model di dalam folder *train* dan *val*

In [ ]:
def copyLabels(labelList, labelpath,folder_Name):
    for label in labelList:
        text = open(labelpath+'/'+label).read()
        with open("./yolov5/data/"+folder_Name+"/labels/"+label, "w") as file:
          file.write(text) 

copyLabels(train_label, path_labels_train,"train")
copyLabels(val_label, path_labels_val,"val")

## Membangun, Melatih, dan Menganalisa Model

Untuk memulai pelatihan model, kita akan pindahkan direktori pekerjaan ke direktori model YOLOv5.

In [ ]:
%cd yolov5

/content/yolov5


Lalu kita akan persiapkan masukan dan keluaran dari model yang akan kita latih nanti.

In [ ]:
from IPython.display import Image, Video, clear_output  # to display images
import torch
from yolov5 import utils
display = utils.notebook_init()

YOLOv5 🚀 v6.2-164-g2787ad7 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 40.8/78.2 GB disk)


Kemudian kita akan buat file `.yaml` yang berisi *brand* yang akan dideteksi.

In [ ]:
yaml_text = """train: data/train/images
val: data/train/images

nc: 38
names: ["Republic of Gamers", "Hello Fresh", "Displate", "KiwiCo", "World of Tanks", "Dollar Shave Club", "SkillShare", "Manscaped", "Rhinoshield", "Raid shadow legends", "Worlds of Warships", "Fruitz", "War Thunder", "Redbull", "Squarespace", "Brilliant.org", "Logitech", "DBrand", "Honey coupon", "Gorillas brand", "levlup", "Ridge wallet", "ExpressVPN", "State of Survival", "Coca Cola", "Crunchyroll", "Uber Eats", "Surfshark", "Corsair", "Lootcrate", "Amazon", "audible", "NordVPN", "GFuel", "Genshin Impact", "TunnelBear VPN", "Microsoft", "Winamax"]"""

with open("data/datacst.yaml", 'w') as file:
    file.write(yaml_text)

%cat data/datacst.yaml

Lalu kita akan modifikasi iPython agar dapat menulis variabel.

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

Kemudian kita akan mengatur arsitektur model YOLOv5 sebagai berikut.

In [ ]:
%%writetemplate models/custom_yolov5s.yaml

# parameters
nc: 38  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
    - [10,13, 16,30, 33,23]  # P3/8
    - [30,61, 62,45, 59,119]  # P4/16
    - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
    [[-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, 'nearest']],
    [[-1, 6], 1, Concat, [1]],  # cat backbone P4
    [-1, 3, BottleneckCSP, [512, False]],  # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, 'nearest']],
    [[-1, 4], 1, Concat, [1]],  # cat backbone P3
    [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

    [-1, 1, Conv, [256, 3, 2]],
    [[-1, 14], 1, Concat, [1]],  # cat head P4
    [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

    [-1, 1, Conv, [512, 3, 2]],
    [[-1, 10], 1, Concat, [1]],  # cat head P5
    [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

    [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
    ]

Lalu kita akan latih model tersebut (Perhatian: Perintah di bawah akan memakan waktu sekitar 4 jam).

In [ ]:
!python train.py --img 640 --batch 64 --epochs 50 --data data/datacst.yaml --cfg models/custom_yolov5s.yaml --weights yolov5s.pt --name yolov5x_fold0

train: weights=yolov5s.pt, cfg=models/custom_yolov5s.yaml, data=data/datacst.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5x_fold0, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/ultralytics/yolov5
   a9f6885..2629dec  update/inference -> origin/update/inference
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-164-g

Dengan pelatihan model sebanyak 50 *epoch*, kita mendapati nilai presisi dan *recall* dari setiap model yang dilatih sudah berada pada rentang 90-100%. Meskipun begitu, akurasi rata-rata model untuk menentukan *brand* tertentu dengan interval yang sangat tinggi masih berada pada rentang 80%-90%. Model ini sudah merupakan model yang *good-fit* terhadap *dataset* yang kita gunakan, namun model ini masih bisa diperbaiki dengan menambah jumlah *epoch* yang akan digunakan.

Kemudian kita akan simpan model terbaik yang sudah dilatih.

In [ ]:
!python export.py --weights runs/train/yolov5x_fold0/weights/best.pt --include tflite --img 640

export: data=data/coco128.yaml, weights=['runs/train/yolov5x_fold0/weights/best.pt'], imgsz=[640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tflite']
YOLOv5 🚀 v6.2-164-g2787ad7 Python-3.7.14 torch-1.12.1+cu113 CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7346307 parameters, 0 gradients

PyTorch: starting from runs/train/yolov5x_fold0/weights/best.pt with output shape (1, 25200, 43) (14.4 MB)

TensorFlow SavedModel: starting export with tensorflow 2.8.2...

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
2022-09-25 19:12:22.796598: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] 

## Penyimpanan Model

Untuk keperluan *deployment* baik secara lokal maupun secara *online*, kita akan unduh beban model yang sudah dilatih untuk kemudian diterapkan di tempat lain tanpa harus melatih kembali model tersebut.

In [ ]:
!zip -r /content/weights.zip /content/yolov5/runs/train/yolov5x_fold0/weights/

files.download('/content/weights.zip')
files.download('/content/yolov5/data/datacst.yaml')

  adding: content/yolov5/runs/train/yolov5x_fold0/weights/ (stored 0%)
  adding: content/yolov5/runs/train/yolov5x_fold0/weights/best-fp16.tflite (deflated 8%)
  adding: content/yolov5/runs/train/yolov5x_fold0/weights/best.pt (deflated 9%)
  adding: content/yolov5/runs/train/yolov5x_fold0/weights/best_saved_model/ (stored 0%)
  adding: content/yolov5/runs/train/yolov5x_fold0/weights/best_saved_model/variables/ (stored 0%)
  adding: content/yolov5/runs/train/yolov5x_fold0/weights/best_saved_model/variables/variables.index (deflated 33%)
  adding: content/yolov5/runs/train/yolov5x_fold0/weights/best_saved_model/variables/variables.data-00000-of-00001 (deflated 4%)
  adding: content/yolov5/runs/train/yolov5x_fold0/weights/best_saved_model/assets/ (stored 0%)
  adding: content/yolov5/runs/train/yolov5x_fold0/weights/best_saved_model/saved_model.pb (deflated 11%)
  adding: content/yolov5/runs/train/yolov5x_fold0/weights/last.pt (deflated 9%)


Kemudian kita akan coba lakukan *inferencing* dengan model yang telah kita latih.

In [ ]:
!python detect.py --source /content/640_data_augment_yolo_light/images/train/amazon__1.png --weight /content/yolov5/runs/train/yolov5x_fold0/weights/best.pt --data /content/yolov5/data/datacst.yaml --name tes --conf 0.4

detect: weights=['/content/yolov5/runs/train/yolov5x_fold0/weights/best.pt'], source=/content/640_data_augment_yolo_light/images/train/amazon__1.png, data=/content/yolov5/data/datacst.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=tes, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-164-g2787ad7 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7346307 parameters, 0 gradients
image 1/1 /content/640_data_augment_yolo_light/images/train/amazon__1.png: 640x640 1 Amazon, 14.2ms
Speed: 0.6ms pre-process, 14.2ms inference, 1.4ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/tes


Hasil *inference* dapat kita lihat di direktori yang tertera di baris terakhir eksekusi perintah tersebut.

Berikut adalah langkah-langkah untuk langsung melakukan *inference* secara lokal secara singkat:

1. Impor library YOLOv5 ke dokumen frontend,

2. Masukkan dokumen yang ada di `weights.zip` ke `/yolov5/runs/train/yolov5x_fold0/weights`

3. Masukkan `datacst.yaml` ke `/yolov5/data`